# PySparkでデータ前処理100本ノックをやってみる

* 参考
    * [PySparkでCSVファイルを読み込む方法](https://docs.kanaries.net/ja/articles/read-csv-dataframe)
    * [PySparkでparquetファイルを読み込む方法](https://data-analysis-stats.jp/spark/pyspark%E3%81%A7%E3%83%87%E3%83%BC%E3%82%BF%E8%AA%AD%E3%81%BF%E8%BE%BC%E3%81%BF/)
    * [PolarsでCSVファイルをParquetに変換する方法](https://medium.com/@umesh.nagar92x/efficient-conversion-of-massive-csv-files-to-parquet-format-using-pandas-dask-duck-db-and-polars-5c998c47b43d)

In [120]:
from glob import glob

import polars as pl
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col
from pyspark.sql import functions as F

# Create a SparkSession。pythonからsparkを使う場合、セッションの作成が必要。
spark = SparkSession.builder.appName("Testing PySpark Example").getOrCreate()

# デフォルトのログレベルだと大量にログが出力されるので限定する。
spark.sparkContext.setLogLevel("ERROR")

## Polarsを用いて入力ファイル(csv)をParquetに変換する
PySparkはcsvを読むのが苦手なので、Parquetファイルに変換する

In [28]:
csv_files = glob('../../../100knocks-preprocess/docker/work/data/*.csv')
csv_files

['../../../100knocks-preprocess/docker/work/data/store.csv',
 '../../../100knocks-preprocess/docker/work/data/receipt.csv',
 '../../../100knocks-preprocess/docker/work/data/category.csv',
 '../../../100knocks-preprocess/docker/work/data/product.csv',
 '../../../100knocks-preprocess/docker/work/data/customer.csv',
 '../../../100knocks-preprocess/docker/work/data/geocode.csv']

In [29]:
dtypes = {
    'customer_id': str,
    'gender_cd': str,
    'postal_cd': str,
    'application_store_cd': str,
    'status_cd': str,
    'category_major_cd': str,
    'category_medium_cd': str,
    'category_small_cd': str,
    'product_cd': str,
    'store_cd': str,
    'prefecture_cd': str,
    'tel_no': str,
    'postal_cd': str,
    'street': str,
    'application_date': str,
    'birth_day': pl.Date
}

# それぞれcsvを読み込んでからparquetとして出力
for file in csv_files:
    df = pl.read_csv(file, dtypes=dtypes)

    # fileの絶対パスの拡張子をparquetにして出力
    output_path = file.split(".csv")[0] + ".parquet"
    df.write_parquet(output_path)


## PySparkでParquetファイルを読み込み

In [75]:
# 各データ読み込み
df_receipt = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/receipt.parquet")

# 店舗データ
df_store = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/store.parquet")

# 顧客データ
df_customer = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/customer.parquet")

In [68]:
df_receipt.printSchema()

root
 |-- sales_ymd: long (nullable = true)
 |-- sales_epoch: long (nullable = true)
 |-- store_cd: string (nullable = true)
 |-- receipt_no: long (nullable = true)
 |-- receipt_sub_no: long (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_cd: string (nullable = true)
 |-- quantity: long (nullable = true)
 |-- amount: long (nullable = true)



In [66]:
df_store.printSchema()

root
 |-- store_cd: string (nullable = true)
 |-- store_name: string (nullable = true)
 |-- prefecture_cd: string (nullable = true)
 |-- prefecture: string (nullable = true)
 |-- address: string (nullable = true)
 |-- address_kana: string (nullable = true)
 |-- tel_no: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- floor_area: double (nullable = true)



# 100本ノック

---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [44]:
df_receipt.show(10)

+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
|sales_ymd|sales_epoch|store_cd|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
| 20181103| 1541203200|  S14006|       112|             1|CS006214000001|P070305012|       1|   158|
| 20181118| 1542499200|  S13008|      1132|             2|CS008415000097|P070701017|       1|    81|
| 20170712| 1499817600|  S14028|      1102|             1|CS028414000014|P060101005|       1|   170|
| 20190205| 1549324800|  S14042|      1132|             1|ZZ000000000000|P050301001|       1|    25|
| 20180821| 1534809600|  S14025|      1102|             2|CS025415000050|P060102007|       1|    90|
| 20190605| 1559692800|  S13003|      1112|             1|CS003515000195|P050102002|       1|   138|
| 20181205| 1543968000|  S14024|      1102|             2|CS024514000042|P080101005|       

---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [45]:
df_receipt.select(
    "sales_ymd",
    "customer_id",
    "product_cd",
    "amount"
).show(10)

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20181103|CS006214000001|P070305012|   158|
| 20181118|CS008415000097|P070701017|    81|
| 20170712|CS028414000014|P060101005|   170|
| 20190205|ZZ000000000000|P050301001|    25|
| 20180821|CS025415000050|P060102007|    90|
| 20190605|CS003515000195|P050102002|   138|
| 20181205|CS024514000042|P080101005|    30|
| 20190922|CS040415000178|P070501004|   128|
| 20170504|ZZ000000000000|P071302010|   770|
| 20191010|CS027514000015|P071101003|   680|
+---------+--------------+----------+------+
only showing top 10 rows



---
> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、  
> 10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更しながら抽出すること。

In [48]:
df_receipt.select(
    F.col("sales_ymd").alias("sales_date"),
    "customer_id",
    "product_cd",
    "amount"
).show(10)

+----------+--------------+----------+------+
|sales_date|   customer_id|product_cd|amount|
+----------+--------------+----------+------+
|  20181103|CS006214000001|P070305012|   158|
|  20181118|CS008415000097|P070701017|    81|
|  20170712|CS028414000014|P060101005|   170|
|  20190205|ZZ000000000000|P050301001|    25|
|  20180821|CS025415000050|P060102007|    90|
|  20190605|CS003515000195|P050102002|   138|
|  20181205|CS024514000042|P080101005|    30|
|  20190922|CS040415000178|P070501004|   128|
|  20170504|ZZ000000000000|P071302010|   770|
|  20191010|CS027514000015|P071101003|   680|
+----------+--------------+----------+------+
only showing top 10 rows



---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [51]:
df_receipt.select(
    "sales_ymd",
    "customer_id",
    "product_cd",
    "amount"
).filter(
    F.col("customer_id") == "CS018205000001"
).show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401012|  2200|
| 20180414|CS018205000001|P060104007|   600|
| 20170614|CS018205000001|P050206001|   990|
| 20170614|CS018205000001|P060702015|   108|
| 20190216|CS018205000001|P071005024|   102|
| 20180414|CS018205000001|P071101002|   278|
| 20190226|CS018205000001|P070902035|   168|
| 20190924|CS018205000001|P060805001|   495|
| 20190226|CS018205000001|P071401020|  2200|
| 20180911|CS018205000001|P071401005|  1100|
| 20190216|CS018205000001|P040101002|   218|
| 20190924|CS018205000001|P091503001|   280|
+---------+--------------+----------+------+



---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、  
> 売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [53]:
df_receipt.select(
    "sales_ymd",
    "customer_id",
    "product_cd",
    "amount"
).filter(
    (F.col("customer_id") == "CS018205000001") & (F.col("amount") >= 1000 )
).show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401012|  2200|
| 20190226|CS018205000001|P071401020|  2200|
| 20180911|CS018205000001|P071401005|  1100|
+---------+--------------+----------+------+



---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、  
> 以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [59]:
cols = ["sales_ymd","customer_id","product_cd","quantity","amount"]
df_receipt.select(
    cols
).filter(
    (F.col("customer_id") == "CS018205000001") &
    ((F.col("amount") >= 1000) | (F.col("quantity") >= 5))
).show()

+---------+--------------+----------+--------+------+
|sales_ymd|   customer_id|product_cd|quantity|amount|
+---------+--------------+----------+--------+------+
| 20180911|CS018205000001|P071401012|       1|  2200|
| 20180414|CS018205000001|P060104007|       6|   600|
| 20170614|CS018205000001|P050206001|       5|   990|
| 20190226|CS018205000001|P071401020|       1|  2200|
| 20180911|CS018205000001|P071401005|       1|  1100|
+---------+--------------+----------+--------+------+



---
> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、  
> 以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [60]:
cols = ["sales_ymd","customer_id","product_cd","amount"]
df_receipt.select(
    cols
).filter(
    (F.col("customer_id") == "CS018205000001") &
    (F.col("amount").between(1000, 2000))
).show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401005|  1100|
+---------+--------------+----------+------+



---
> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、  
> 以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [61]:
cols = ["sales_ymd","customer_id","product_cd","amount"]
df_receipt.select(
    cols
).filter(
    (F.col("customer_id") == "CS018205000001") &
    (F.col("product_cd") != "P071401019")
).show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401012|  2200|
| 20180414|CS018205000001|P060104007|   600|
| 20170614|CS018205000001|P050206001|   990|
| 20170614|CS018205000001|P060702015|   108|
| 20190216|CS018205000001|P071005024|   102|
| 20180414|CS018205000001|P071101002|   278|
| 20190226|CS018205000001|P070902035|   168|
| 20190924|CS018205000001|P060805001|   495|
| 20190226|CS018205000001|P071401020|  2200|
| 20180911|CS018205000001|P071401005|  1100|
| 20190216|CS018205000001|P040101002|   218|
| 20190924|CS018205000001|P091503001|   280|
+---------+--------------+----------+------+



---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
> 
> `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [70]:
# 問題文の結果
df_store.filter(
    ~((F.col("prefecture_cd") == "13") | (F.col("floor_area") > 900))
).show()

+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+---------+--------+----------+
|store_cd|store_name|prefecture_cd|prefecture|                           address|                          address_kana|      tel_no|longitude|latitude|floor_area|
+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+---------+--------+----------+
|  S14046|  北山田店|           14|  神奈川県|  神奈川県横浜市都筑区北山田一丁目| カナガワケンヨコハマシツヅキクキタ...|045-123-4049| 139.5916|35.56189|     831.0|
|  S14011|日吉本町店|           14|  神奈川県|神奈川県横浜市港北区日吉本町四丁目| カナガワケンヨコハマシコウホククヒ...|045-123-4033| 139.6316|35.54655|     890.0|
|  S12013|  習志野店|           12|    千葉県|          千葉県習志野市芝園一丁目|チバケンナラシノシシバゾノイッチョウメ|047-123-4002|  140.022|35.66122|     808.0|
+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+-----

In [71]:
# 書き換え
df_store.filter(
    ((F.col("prefecture_cd") != "13") & (F.col("floor_area") <= 900))
).show()

+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+---------+--------+----------+
|store_cd|store_name|prefecture_cd|prefecture|                           address|                          address_kana|      tel_no|longitude|latitude|floor_area|
+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+---------+--------+----------+
|  S14046|  北山田店|           14|  神奈川県|  神奈川県横浜市都筑区北山田一丁目| カナガワケンヨコハマシツヅキクキタ...|045-123-4049| 139.5916|35.56189|     831.0|
|  S14011|日吉本町店|           14|  神奈川県|神奈川県横浜市港北区日吉本町四丁目| カナガワケンヨコハマシコウホククヒ...|045-123-4033| 139.6316|35.54655|     890.0|
|  S12013|  習志野店|           12|    千葉県|          千葉県習志野市芝園一丁目|チバケンナラシノシシバゾノイッチョウメ|047-123-4002|  140.022|35.66122|     808.0|
+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+-----

---
> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [74]:
df_store.filter(
    F.col("store_cd").startswith("S14")
).show(10)

+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|store_cd|  store_name|prefecture_cd|prefecture|                               address|                            address_kana|      tel_no|longitude|latitude|floor_area|
+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|  S14010|      菊名店|           14|  神奈川県|        神奈川県横浜市港北区菊名一丁目|   カナガワケンヨコハマシコウホククキ...|045-123-4032| 139.6326|35.50049|    1732.0|
|  S14033|    阿久和店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...|045-123-4043| 139.4961|35.45918|    1495.0|
|  S14036|相模原中央店|           14|  神奈川県|            神奈川県相模原市中央二丁目|   カナガワケンサガミハラシチュウオウ...|042-123-4045| 139.3716|35.57327|    1679.0|
|  S14040|    長津田店|           14|  神奈川県|神奈川県横浜市緑区長津田みなみ台五丁目|   カナガワケンヨコハマシミドリクナガ...|045-1

---
> P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [77]:
df_customer.filter(
    F.col("customer_id").endswith("1")
).show(10)

+--------------+-------------+---------+------+----------+---+---------+---------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                          address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+---------------------------------+--------------------+----------------+------------+
|CS037613000071|    六角 雅彦|        9|  不明|1952-04-01| 66| 136-0076|       東京都江東区南砂**********|              S13037|        20150414|0-00000000-0|
|CS028811000001|  堀井 かおり|        1|  女性|1933-03-27| 86| 245-0016| 神奈川県横浜市泉区和泉町*****...|              S14028|        20160115|0-00000000-0|
|CS040412000191|    川井 郁恵|        1|  女性|1977-01-05| 42| 226-0021|神奈川県横浜市緑区北八朔町****...|              S14040|        20151101|1-20091025-4|
|CS028314000011|  小菅 あおい|        1|  女性|1983-11-26| 35| 246-0038| 神奈川県横浜市瀬谷区宮沢*****...|              S14

---
> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [79]:
df_store.filter(
    F.col("address").contains("横浜市")
).show()

+--------+----------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|store_cd|store_name|prefecture_cd|prefecture|                               address|                            address_kana|      tel_no|longitude|latitude|floor_area|
+--------+----------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|  S14010|    菊名店|           14|  神奈川県|        神奈川県横浜市港北区菊名一丁目|   カナガワケンヨコハマシコウホククキ...|045-123-4032| 139.6326|35.50049|    1732.0|
|  S14033|  阿久和店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...|045-123-4043| 139.4961|35.45918|    1495.0|
|  S14040|  長津田店|           14|  神奈川県|神奈川県横浜市緑区長津田みなみ台五丁目|   カナガワケンヨコハマシミドリクナガ...|045-123-4046| 139.4994|35.52398|    1548.0|
|  S14050|阿久和西店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...|045-123-4053| 139.4961

---
> P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [86]:
# 1. F.regexpを使う方法。F.litで正規表現の列表現を指定する必要がある。
df_customer.filter(
    F.regexp("status_cd", F.lit("^[A-F]"))
).show(10)

+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                           address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|CS031415000172|宇多田 貴美子|        1|  女性|1976-10-04| 42| 151-0053|      東京都渋谷区代々木**********|              S13031|        20150529|D-20100325-C|
|CS015414000103|    奥野 陽子|        1|  女性|1977-08-09| 41| 136-0073|        東京都江東区北砂**********|              S13015|        20150722|B-20100609-B|
|CS011215000048|    芦田 沙耶|        1|  女性|1992-02-01| 27| 223-0062|神奈川県横浜市港北区日吉本町***...|              S14011|        20150228|C-20100421-9|
|CS029415000023|    梅田 里穂|        1|  女性|1976-01-17| 43| 279-0043|      千葉県浦安市富士見**********|     

In [88]:
# 2. SQLっぽいマッチのさせ方。こっちの方が簡単に書ける。
df_customer.filter(
    F.col("status_cd").rlike("^[A-F]")
).show(10)

+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                           address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|CS031415000172|宇多田 貴美子|        1|  女性|1976-10-04| 42| 151-0053|      東京都渋谷区代々木**********|              S13031|        20150529|D-20100325-C|
|CS015414000103|    奥野 陽子|        1|  女性|1977-08-09| 41| 136-0073|        東京都江東区北砂**********|              S13015|        20150722|B-20100609-B|
|CS011215000048|    芦田 沙耶|        1|  女性|1992-02-01| 27| 223-0062|神奈川県横浜市港北区日吉本町***...|              S14011|        20150228|C-20100421-9|
|CS029415000023|    梅田 里穂|        1|  女性|1976-01-17| 43| 279-0043|      千葉県浦安市富士見**********|     

In [93]:
# containsではうまくまっちしない
df_customer.filter(
    F.col("status_cd").contains(r"^[A-F]")
).show(10)

+-----------+-------------+---------+------+---------+---+---------+-------+--------------------+----------------+---------+
|customer_id|customer_name|gender_cd|gender|birth_day|age|postal_cd|address|application_store_cd|application_date|status_cd|
+-----------+-------------+---------+------+---------+---+---------+-------+--------------------+----------------+---------+
+-----------+-------------+---------+------+---------+---+---------+-------+--------------------+----------------+---------+



---
> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [91]:
df_customer.filter(
    F.col("status_cd").rlike("[1-9]$")
).show(10)

+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                           address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|CS001215000145|    田崎 美紀|        1|  女性|1995-03-29| 24| 144-0055|      東京都大田区仲六郷**********|              S13001|        20170605|6-20090929-2|
|CS033513000180|      安斎 遥|        1|  女性|1962-07-11| 56| 241-0823|  神奈川県横浜市旭区善部町*****...|              S14033|        20150728|6-20080506-5|
|CS011215000048|    芦田 沙耶|        1|  女性|1992-02-01| 27| 223-0062|神奈川県横浜市港北区日吉本町***...|              S14011|        20150228|C-20100421-9|
|CS040412000191|    川井 郁恵|        1|  女性|1977-01-05| 42| 226-0021| 神奈川県横浜市緑区北八朔町****...|          

---
> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、  
> 末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [96]:
df_customer.filter(
    (F.col("status_cd").rlike("^[A-F].*[1-9]$")) 
).show(10)

+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                           address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|CS011215000048|    芦田 沙耶|        1|  女性|1992-02-01| 27| 223-0062|神奈川県横浜市港北区日吉本町***...|              S14011|        20150228|C-20100421-9|
|CS022513000105|  島村 貴美子|        1|  女性|1962-03-12| 57| 249-0002|    神奈川県逗子市山の根**********|              S14022|        20150320|A-20091115-7|
|CS001515000096|    水野 陽子|        9|  不明|1960-11-29| 58| 144-0053|    東京都大田区蒲田本町**********|              S13001|        20150614|A-20100724-7|
|CS013615000053|    西脇 季衣|        1|  女性|1953-10-18| 65| 261-0026|  千葉県千葉市美浜区幕張西*****...|          

---
> P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [110]:
df_store.filter(
    F.col("tel_no").rlike("^[0-9]{3}-[0-9]{3}-[0-9]{4}$")
).show()

+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|store_cd|  store_name|prefecture_cd|prefecture|                               address|                            address_kana|      tel_no|longitude|latitude|floor_area|
+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|  S12014|    千草台店|           12|    千葉県|        千葉県千葉市稲毛区千草台一丁目|   チバケンチバシイナゲクチグサダイイ...|043-123-4003|  140.118|35.63559|    1698.0|
|  S13002|    国分寺店|           13|    東京都|              東京都国分寺市本多二丁目|トウキョウトコクブンジシホンダニチョウメ|042-123-4008| 139.4802|35.70566|    1735.0|
|  S14010|      菊名店|           14|  神奈川県|        神奈川県横浜市港北区菊名一丁目|   カナガワケンヨコハマシコウホククキ...|045-123-4032| 139.6326|35.50049|    1732.0|
|  S14033|    阿久和店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...

---
> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [116]:
df_customer.sort("birth_day").show(10)

+--------------+-------------+---------+------+----------+---+---------+--------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                         address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+--------------------------------+--------------------+----------------+------------+
|CS003813000014|  村山 菜々美|        1|  女性|1928-11-26| 90| 182-0007|    東京都調布市菊野台**********|              S13003|        20160214|0-00000000-0|
|CS026813000004|    吉村 朝陽|        1|  女性|1928-12-14| 90| 251-0043| 神奈川県藤沢市辻堂元町******...|              S14026|        20150723|0-00000000-0|
|CS018811000003|    熊沢 美里|        1|  女性|1929-01-07| 90| 204-0004|      東京都清瀬市野塩**********|              S13018|        20150403|0-00000000-0|
|CS027803000004|    内村 拓郎|        0|  男性|1929-01-12| 90| 251-0031|神奈川県藤沢市鵠沼藤が谷*****...|              S140

---
> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [117]:
# 降順にソート
df_customer.sort("birth_day", ascending=False).show(10)

+--------------+-------------+---------+------+----------+---+---------+---------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                          address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+---------------------------------+--------------------+----------------+------------+
|CS035114000004|    大村 美里|        1|  女性|2007-11-25| 11| 156-0053|       東京都世田谷区桜**********|              S13035|        20150619|6-20091205-6|
|CS022103000002|  福山 はじめ|        9|  不明|2007-10-02| 11| 249-0006|     神奈川県逗子市逗子**********|              S14022|        20160909|0-00000000-0|
|CS002113000009|  柴田 真悠子|        1|  女性|2007-09-17| 11| 184-0014|  東京都小金井市貫井南町******...|              S13002|        20160304|0-00000000-0|
|CS004115000014|    松井 京子|        1|  女性|2007-08-09| 11| 165-0031|     東京都中野区上鷺宮**********|         

---
> P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。  
> 項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [131]:
w = Window.orderBy(F.desc("amount"))

df_receipt.select(
    "customer_id",
    "amount",
    F.dense_rank().over(w).alias("順位")
).show(10)

+--------------+------+----+
|   customer_id|amount|順位|
+--------------+------+----+
|CS011415000006| 10925|   1|
|ZZ000000000000|  6800|   2|
|CS028605000002|  5780|   3|
|CS015515000034|  5480|   4|
|ZZ000000000000|  5480|   4|
|ZZ000000000000|  5480|   4|
|ZZ000000000000|  5440|   5|
|CS021515000089|  5440|   5|
|ZZ000000000000|  5280|   6|
|CS017414000114|  5280|   6|
+--------------+------+----+
only showing top 10 rows



dense_rankによって同着がいた場合、次の順位は同着＋１の順位になる。そうしたくない場合はrankを用いること。  
使い方はSQLのような感じで.over(window)のように使用する。windowには対象カラムに対するソート操作を定義する。(polarsと異なり、結構めんどくさい。)  
また、pyspark.sqlのWindowクラスをimportしておく必要があることに注意。    
* [dense_rankについて](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.dense_rank.html?highlight=dense_rank)
* [Window関数について](https://www.niandc.co.jp/tech/20220613_2187/)
OVERはこれからWindow関数を使うサインだと考えればよいみたい。

---
> P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。  
> 項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [134]:
w = Window.orderBy(F.desc("amount"))

df_receipt.select(
    "customer_id",
    "amount",
    F.row_number().over(w).alias("順位")
).show()

+--------------+------+----+
|   customer_id|amount|順位|
+--------------+------+----+
|CS011415000006| 10925|   1|
|ZZ000000000000|  6800|   2|
|CS028605000002|  5780|   3|
|CS015515000034|  5480|   4|
|ZZ000000000000|  5480|   5|
|ZZ000000000000|  5480|   6|
|ZZ000000000000|  5440|   7|
|CS021515000089|  5440|   8|
|ZZ000000000000|  5280|   9|
|CS028515000029|  5280|  10|
|ZZ000000000000|  5280|  11|
|CS035615000040|  5280|  12|
|ZZ000000000000|  5280|  13|
|CS030515000189|  5280|  14|
|CS017414000114|  5280|  15|
|ZZ000000000000|  5280|  16|
|CS035414000024|  5280|  17|
|CS039415000073|  5280|  18|
|CS021515000089|  5280|  19|
|CS038415000071|  5280|  20|
+--------------+------+----+
only showing top 20 rows



同着でも構わず連続の順位をつけたい場合はrow_numberを用いる。ここはSQLと一緒。

---
> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [135]:
df_receipt.count()

104681

In [136]:
len(df_receipt)

TypeError: object of type 'DataFrame' has no len()

pysparkのDataframeにはlenは使えないので注意。

---
> P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [163]:
df_receipt.select(
    F.count_distinct("customer_id")
).show()

+---------------------------+
|count(DISTINCT customer_id)|
+---------------------------+
|                       8307|
+---------------------------+



polarsのuniqueではなくcount_distinctでユニークな件数をカウントすることに注意

In [164]:
# 単一の数字としてほしい場合は一度collectしてから下記のように取り出す。
df_receipt.select(
    F.count_distinct("customer_id")
).collect()[0]["count(DISTINCT customer_id)"]

8307